In [1]:
import glob
import numpy as np
import cv2

In [2]:
# reading data
patches = glob.glob('./data/**/*.png', recursive=True)

In [3]:
def pad_data():
    """ Pads images that are smaller than 50x50 with zeros
        Saves images in a new folder
    """
    WIDTH = 50
    HEIGHT = 50
    for img_name in patches:
        image = cv2.imread(img_name)
        orig_height, orig_width, _ = image.shape
        path = "./padded/" + img_name[14:]
        if (orig_height != 50 or orig_width != 50):
            height_diff = HEIGHT - orig_height
            width_diff = WIDTH - orig_width
            top = height_diff // 2 # floor division
            bottom = height_diff - top
            left = width_diff // 2
            right = width_diff - left
            constant = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
            cv2.imwrite(path, constant)
        else:
            cv2.imwrite(path, image)

In [5]:
pad_data()